In [5]:
!pip install scikit-learn

  Obtaining dependency information for scikit-learn from https://files.pythonhosted.org/packages/c6/99/ed35197a158f1fdc2fe7c3680e9c70d0128f662e1fee4ed495f4b5e13db0/scikit_learn-1.7.2-cp312-cp312-win_amd64.whl.metadata
  Obtaining dependency information for threadpoolctl>=3.1.0 from https://files.pythonhosted.org/packages/32/d5/f9a850d79b0851d1d4ef6456097579a9005b31fea68726a4ae5f2d82ddd9/threadpoolctl-3.6.0-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/8.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/8.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/8.7 MB 640.0 kB/s eta 0:00:14
   ---------------------------------------- 0.1/8.7 MB 648.1 kB/s eta 0:00:14
    --------------------------------------- 0.2/8.7 MB 1.7 MB/s eta 0:00:06
   - -------------------------------------- 0.4/8.7 MB 2.4 MB/s eta 0:00:04
   -- ------------------------------------- 0.6/8.7 MB 3.0 MB/s eta 0:00:03
   --- -----------------------------------


[notice] A new release of pip is available: 23.2.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import cv2
import mediapipe as mp
import numpy as np
from ultralytics import YOLO
import pandas as pd
import cv2
import mediapipe as mp
import numpy as np
from ultralytics import YOLO
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
import joblib
import time

# -------------------- Initialize models --------------------
PHONE_CLASSES = ["cell phone", "mobile phone"]
phone_model = YOLO("yolov8n.pt")

face_mesh = mp.solutions.face_mesh.FaceMesh(refine_landmarks=True, max_num_faces=1, min_detection_confidence=0.5)

POSE_LANDMARKS = [1, 33, 263, 61, 291, 199]
LEFT_EYE = [33, 133, 159, 145]
LEFT_IRIS = [468, 469, 470, 471]
RIGHT_EYE = [362, 263, 386, 374]
RIGHT_IRIS = [473, 474, 475, 476]

# -------------------- Smoothers --------------------
class TwoDimSmoother:
    def __init__(self, alpha=0.3):
        self.alpha = alpha
        self.x = None
        self.y = None
    def apply(self, newx, newy):
        if self.x is None:
            self.x, self.y = newx, newy
        else:
            self.x = (1 - self.alpha) * self.x + self.alpha * newx
            self.y = (1 - self.alpha) * self.y + self.alpha * newy
        return int(self.x), int(self.y)

class HeadPoseSmoother:
    def __init__(self, alpha=0.2):
        self.alpha = alpha
        self.pitch = None
        self.yaw = None
        self.roll = None
    def apply(self, pitch, yaw, roll):
        if self.pitch is None:
            self.pitch, self.yaw, self.roll = pitch, yaw, roll
        else:
            self.pitch = (1 - self.alpha) * self.pitch + self.alpha * pitch
            self.yaw   = (1 - self.alpha) * self.yaw   + self.alpha * yaw
            self.roll  = (1 - self.alpha) * self.roll  + self.alpha * roll
        return self.pitch, self.yaw, self.roll

kalman_left = TwoDimSmoother()
kalman_right = TwoDimSmoother()
head_smoother = HeadPoseSmoother()

# -------------------- Eye direction --------------------
def get_eye_direction(landmarks, eye_idx, iris_idx, w, h, smoother):
    eye = [(int(landmarks[i].x * w), int(landmarks[i].y * h)) for i in eye_idx]
    iris_pts = [(int(landmarks[i].x * w), int(landmarks[i].y * h)) for i in iris_idx]
    iris_cx = sum([p[0] for p in iris_pts]) / len(iris_pts)
    iris_cy = sum([p[1] for p in iris_pts]) / len(iris_pts)
    iris_x, iris_y = smoother.apply(iris_cx, iris_cy)

    left, right = min(eye[0][0], eye[1][0]), max(eye[0][0], eye[1][0])
    top, bottom = min(eye[2][1], eye[3][1]), max(eye[2][1], eye[3][1])

    margin_x = int((right - left) * 0.15)
    margin_y = int((bottom - top) * 0.20)

    # Determine gaze direction and offsets
    if iris_x < left + margin_x:
        return "LEFT", iris_x - (left + margin_x), iris_y - (top + margin_y)
    elif iris_x > right - margin_x:
        return "RIGHT", iris_x - (left + margin_x), iris_y - (top + margin_y)
    elif iris_y < top + margin_y:
        return "UP", iris_x - (left + margin_x), iris_y - (top + margin_y)
    elif iris_y > bottom - margin_y:
        return "DOWN", iris_x - (left + margin_x), iris_y - (top + margin_y)
    else:
        return "CENTER", iris_x - (left + margin_x), iris_y - (top + margin_y)

# -------------------- Head pose --------------------
def estimate_head_pose(landmarks, w, h):
    image_points = np.array([(landmarks[idx].x * w, landmarks[idx].y * h) for idx in POSE_LANDMARKS], dtype="double")
    model_points = np.array([
        (0.0, 0.0, 0.0),
        (-30.0, 0.0, -30.0),
        (30.0, 0.0, -30.0),
        (-30.0, 0.0, -90.0),
        (30.0, 0.0, -90.0),
        (0.0, 40.0, -50.0)
    ])
    focal_length = w
    center = (w / 2, h / 2)
    camera_matrix = np.array([[focal_length,0,center[0]],[0,focal_length,center[1]],[0,0,1]])
    dist_coeffs = np.zeros((4,1))
    success, rotation_vector, _ = cv2.solvePnP(model_points, image_points, camera_matrix, dist_coeffs, flags=cv2.SOLVEPNP_ITERATIVE)
    return rotation_vector if success else None

# -------------------- Collect data --------------------
cap = cv2.VideoCapture(0)
dir_map = {"LEFT":0,"RIGHT":1,"UP":2,"DOWN":3,"CENTER":4}

columns = ["left_dir","right_dir","left_dx","left_dy","right_dx","right_dy",
           "pitch","yaw","roll","phone_detected","eyes_away_duration","label"]
data_rows = []

away_start_time = None
AWAY_THRESHOLD = 0.8

print("Press 0 for FOCUSED, 1 for DISTRACTED, ESC to quit and train model.")

while True:
    ret, frame = cap.read()
    if not ret: break
    frame = cv2.flip(frame,1)
    h,w,_ = frame.shape
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    eyes_away = False
    phone_detected = 0

    # Face and eyes
    face_results = face_mesh.process(rgb)
    left_dir = right_dir = "CENTER"
    left_dx = left_dy = right_dx = right_dy = 0
    pitch = yaw = roll = 0
    eyes_away_duration = 0

    if face_results.multi_face_landmarks:
        landmarks = face_results.multi_face_landmarks[0].landmark
        left_dir, left_dx, left_dy = get_eye_direction(landmarks, LEFT_EYE, LEFT_IRIS, w, h, kalman_left)
        right_dir, right_dx, right_dy = get_eye_direction(landmarks, RIGHT_EYE, RIGHT_IRIS, w, h, kalman_right)
        rot_vec = estimate_head_pose(landmarks, w, h)
        if rot_vec is not None:
            pitch, yaw, roll = head_smoother.apply(*rot_vec.ravel())
        if left_dir!="CENTER" or right_dir!="CENTER":
            eyes_away = True

    # Phone detection
    results = phone_model(frame, verbose=False)
    for box in results[0].boxes:
        cls = int(box.cls[0])
        label = phone_model.names[cls]
        conf = float(box.conf[0])
        if label in PHONE_CLASSES and conf>0.5:
            phone_detected = 1

    # Eye-away timer
    current_time = time.time()
    if eyes_away:
        if away_start_time is None: away_start_time=current_time
        eyes_away_duration = current_time - away_start_time
    else:
        away_start_time=None
        eyes_away_duration=0

    # Display frame
    cv2.putText(frame,"Press 0:FOCUSED 1:DISTRACTED",(10,30),cv2.FONT_HERSHEY_SIMPLEX,0.7,(0,255,255),2)
    cv2.imshow("Data Collection", frame)

    # Wait for key
    key = cv2.waitKey(1)
    if key==27: break  # ESC to quit
    label = None
    if key==ord("0"): label=0
    elif key==ord("1"): label=1
    if label is not None:
        row = [dir_map[left_dir], dir_map[right_dir], left_dx, left_dy, right_dx, right_dy,
               pitch, yaw, roll, phone_detected, eyes_away_duration, label]
        data_rows.append(row)

cap.release()
cv2.destroyAllWindows()

# -------------------- Train model --------------------
df = pd.DataFrame(data_rows, columns=columns)
X = df.drop("label", axis=1)
y = df["label"]

clf = RandomForestClassifier(n_estimators=100)
clf.fit(X, y)

joblib.dump(clf, "gaze_model.pkl")
print("Model trained and saved as gaze_model.pkl!")

import joblib
import time

# -------------------- Initialize models --------------------
PHONE_CLASSES = ["cell phone", "mobile phone"]
phone_model = YOLO("yolov8n.pt")

face_mesh = mp.solutions.face_mesh.FaceMesh(refine_landmarks=True, max_num_faces=1, min_detection_confidence=0.5)

POSE_LANDMARKS = [1, 33, 263, 61, 291, 199]
LEFT_EYE = [33, 133, 159, 145]
LEFT_IRIS = [468, 469, 470, 471]
RIGHT_EYE = [362, 263, 386, 374]
RIGHT_IRIS = [473, 474, 475, 476]

# -------------------- Smoothers --------------------
class TwoDimSmoother:
    def __init__(self, alpha=0.3):
        self.alpha = alpha
        self.x = None
        self.y = None
    def apply(self, newx, newy):
        if self.x is None:
            self.x, self.y = newx, newy
        else:
            self.x = (1 - self.alpha) * self.x + self.alpha * newx
            self.y = (1 - self.alpha) * self.y + self.alpha * newy
        return int(self.x), int(self.y)

class HeadPoseSmoother:
    def __init__(self, alpha=0.2):
        self.alpha = alpha
        self.pitch = None
        self.yaw = None
        self.roll = None
    def apply(self, pitch, yaw, roll):
        if self.pitch is None:
            self.pitch, self.yaw, self.roll = pitch, yaw, roll
        else:
            self.pitch = (1 - self.alpha) * self.pitch + self.alpha * pitch
            self.yaw   = (1 - self.alpha) * self.yaw   + self.alpha * yaw
            self.roll  = (1 - self.alpha) * self.roll  + self.alpha * roll
        return self.pitch, self.yaw, self.roll

kalman_left = TwoDimSmoother()
kalman_right = TwoDimSmoother()
head_smoother = HeadPoseSmoother()

# -------------------- Eye direction --------------------
def get_eye_direction(landmarks, eye_idx, iris_idx, w, h, smoother):
    eye = [(int(landmarks[i].x * w), int(landmarks[i].y * h)) for i in eye_idx]
    iris_pts = [(int(landmarks[i].x * w), int(landmarks[i].y * h)) for i in iris_idx]
    iris_cx = sum([p[0] for p in iris_pts]) / len(iris_pts)
    iris_cy = sum([p[1] for p in iris_pts]) / len(iris_pts)
    iris_x, iris_y = smoother.apply(iris_cx, iris_cy)

    left, right = min(eye[0][0], eye[1][0]), max(eye[0][0], eye[1][0])
    top, bottom = min(eye[2][1], eye[3][1]), max(eye[2][1], eye[3][1])

    margin_x = int((right - left) * 0.15)
    margin_y = int((bottom - top) * 0.20)

    # Determine gaze direction and offsets
    if iris_x < left + margin_x:
        return "LEFT", iris_x - (left + margin_x), iris_y - (top + margin_y)
    elif iris_x > right - margin_x:
        return "RIGHT", iris_x - (left + margin_x), iris_y - (top + margin_y)
    elif iris_y < top + margin_y:
        return "UP", iris_x - (left + margin_x), iris_y - (top + margin_y)
    elif iris_y > bottom - margin_y:
        return "DOWN", iris_x - (left + margin_x), iris_y - (top + margin_y)
    else:
        return "CENTER", iris_x - (left + margin_x), iris_y - (top + margin_y)

# -------------------- Head pose --------------------
def estimate_head_pose(landmarks, w, h):
    image_points = np.array([(landmarks[idx].x * w, landmarks[idx].y * h) for idx in POSE_LANDMARKS], dtype="double")
    model_points = np.array([
        (0.0, 0.0, 0.0),
        (-30.0, 0.0, -30.0),
        (30.0, 0.0, -30.0),
        (-30.0, 0.0, -90.0),
        (30.0, 0.0, -90.0),
        (0.0, 40.0, -50.0)
    ])
    focal_length = w
    center = (w / 2, h / 2)
    camera_matrix = np.array([[focal_length,0,center[0]],[0,focal_length,center[1]],[0,0,1]])
    dist_coeffs = np.zeros((4,1))
    success, rotation_vector, _ = cv2.solvePnP(model_points, image_points, camera_matrix, dist_coeffs, flags=cv2.SOLVEPNP_ITERATIVE)
    return rotation_vector if success else None

# -------------------- Collect data --------------------
cap = cv2.VideoCapture(0)
dir_map = {"LEFT":0,"RIGHT":1,"UP":2,"DOWN":3,"CENTER":4}

columns = ["left_dir","right_dir","left_dx","left_dy","right_dx","right_dy",
           "pitch","yaw","roll","phone_detected","eyes_away_duration","label"]
data_rows = []

away_start_time = None
AWAY_THRESHOLD = 0.8

print("Press 0 for FOCUSED, 1 for DISTRACTED, ESC to quit and train model.")

while True:
    ret, frame = cap.read()
    if not ret: break
    frame = cv2.flip(frame,1)
    h,w,_ = frame.shape
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    eyes_away = False
    phone_detected = 0

    # Face and eyes
    face_results = face_mesh.process(rgb)
    left_dir = right_dir = "CENTER"
    left_dx = left_dy = right_dx = right_dy = 0
    pitch = yaw = roll = 0
    eyes_away_duration = 0

    if face_results.multi_face_landmarks:
        landmarks = face_results.multi_face_landmarks[0].landmark
        left_dir, left_dx, left_dy = get_eye_direction(landmarks, LEFT_EYE, LEFT_IRIS, w, h, kalman_left)
        right_dir, right_dx, right_dy = get_eye_direction(landmarks, RIGHT_EYE, RIGHT_IRIS, w, h, kalman_right)
        rot_vec = estimate_head_pose(landmarks, w, h)
        if rot_vec is not None:
            pitch, yaw, roll = head_smoother.apply(*rot_vec.ravel())
        if left_dir!="CENTER" or right_dir!="CENTER":
            eyes_away = True

    # Phone detection
    results = phone_model(frame, verbose=False)
    for box in results[0].boxes:
        cls = int(box.cls[0])
        label = phone_model.names[cls]
        conf = float(box.conf[0])
        if label in PHONE_CLASSES and conf>0.5:
            phone_detected = 1

    # Eye-away timer
    current_time = time.time()
    if eyes_away:
        if away_start_time is None: away_start_time=current_time
        eyes_away_duration = current_time - away_start_time
    else:
        away_start_time=None
        eyes_away_duration=0

    # Display frame
    cv2.putText(frame,"Press 0:FOCUSED 1:DISTRACTED",(10,30),cv2.FONT_HERSHEY_SIMPLEX,0.7,(0,255,255),2)
    cv2.imshow("Data Collection", frame)

    # Wait for key
    key = cv2.waitKey(1)
    if key==27: break  # ESC to quit
    label = None
    if key==ord("0"): label=0
    elif key==ord("1"): label=1
    if label is not None:
        row = [dir_map[left_dir], dir_map[right_dir], left_dx, left_dy, right_dx, right_dy,
               pitch, yaw, roll, phone_detected, eyes_away_duration, label]
        data_rows.append(row)

cap.release()
cv2.destroyAllWindows()

# -------------------- Train model --------------------
df = pd.DataFrame(data_rows, columns=columns)
X = df.drop("label", axis=1)
y = df["label"]

clf = RandomForestClassifier(n_estimators=100)
clf.fit(X, y)

joblib.dump(clf, "gaze_model.pkl")
print("Model trained and saved as gaze_model.pkl!")


Press 0 for FOCUSED, 1 for DISTRACTED, ESC to quit and train model.
Model trained and saved as gaze_model.pkl!
Press 0 for FOCUSED, 1 for DISTRACTED, ESC to quit and train model.
Model trained and saved as gaze_model.pkl!
